# Modeling GNNePCSAFT

## Starting point

In [1]:
import os, shutil
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("awskeyid")
secret_value_1 = user_secrets.get_secret("awssecretkey")
secret_value_2 = user_secrets.get_secret("ghtk")
secret_value_3 = user_secrets.get_secret("kaggle")
secret_value_4 = user_secrets.get_secret("username")
secret_value_5 = user_secrets.get_secret("wandbkey")
secret_value_6 = user_secrets.get_secret("email")

os.environ["KAGGLE_USERNAME"] = secret_value_4
os.environ["KAGGLE_KEY"] = secret_value_3
os.environ["WANDB_API_KEY"] = secret_value_5
os.environ["AWS_ACCESS_KEY_ID"] = secret_value_0
os.environ["AWS_SECRET_ACCESS_KEY"] = secret_value_1
os.environ["USER_EMAIL"]=secret_value_6
os.environ["GHTK"]=secret_value_2
os.environ['PATH'] = "/root/.local/bin:" + os.environ['PATH']

In [2]:
%%bash
python -m pip install --upgrade pip ml_collections -q

pip install pipenv --user -q


In [3]:
%%bash
# for the cloud 
rm -rf .* *
git clone --branch dev --single-branch -q https://$GHTK@github.com/wildsonbbl/gnnepcsaft.git .

rm: refusing to remove '.' or '..' directory: skipping '.'


rm: refusing to remove '.' or '..' directory: skipping '..'


In [4]:
%%capture req_output --no-stdout
%%bash
pipenv install -r requirements.txt
echo "*********  Finished pip installing requirements.txt  *********"
pipenv install torch-scatter
echo "*********  Finished pip installing torch-scatter  *********"

# install pcsaft
curl -O -L https://gitlab.com/libeigen/eigen/-/archive/master/eigen-master.zip
curl -O -L https://github.com/zmeri/PC-SAFT/archive/refs/tags/v1.4.1.zip
unzip -q eigen-master.zip
unzip -q v1.4.1.zip
cp -rf eigen-master/. PC-SAFT-1.4.1/externals/eigen
pipenv install ./PC-SAFT-1.4.1
# install pcsaft
echo "*********  Finished pip installing pcsaft  *********"


⠼ Creating virtual environment...


Building requirements...


Resolving dependencies...


✔ Success!


⠸ Locking...


...

⠼ Locking...


Installing dependencies from Pipfile.lock (8cea26)...


To activate this project's virtualenv, run pipenv shell.


Alternatively, run a command inside the virtualenv with pipenv run.


*********  Finished pip installing pcsaft  *********


In [5]:
%%bash

echo AWS_ACCESS_KEY_ID=$AWS_ACCESS_KEY_ID > .env
echo AWS_SECRET_ACCESS_KEY=$AWS_SECRET_ACCESS_KEY >>.env
echo WANDB_API_KEY=$WANDB_API_KEY >> .env

In [6]:
%%bash

pipenv run dvc pull

Loading .env environment variables...


A       gnnepcsaft/data/thermoml/


A       gnnepcsaft/data/ramirez2022/


A       gnnepcsaft/train/checkpoints/


3 files added and 108 files fetched


In [7]:
%%bash
pipenv run pip show pcsaft | grep -E "Version|Name"

lscpu | grep -E "Model name|Core|core" 

nvidia-smi

Loading .env environment variables...


Name: pcsaft


Version: 1.4.1


Thread(s) per core:                 2


Core(s) per socket:                 2


...

|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |


|        ID   ID                                                             Usage      |


|=======================================================================================|


|  No running processes found                                                           |


+---------------------------------------------------------------------------------------+


## Training

In [8]:
from gnnepcsaft.configs.default import get_config
config = get_config()
dataset = config.dataset
model_name = config.model_name
checkpoint = config.checkpoint
model_suffix = str(config.num_train_steps / 1e6).replace(".", "_")
os.environ["DATASET"]=dataset
os.environ["MODEL_NAME"]=model_name
os.environ["MODEL_SUFFIX"]=model_suffix

In [9]:
# if checkpoint:
#     shutil.copy(f"gnnepcsaft/train/checkpoints/{checkpoint}", "gnnepcsaft/train/checkpoints/last_checkpoint.pth")

In [10]:
%%bash
pipenv run python -m gnnepcsaft.train.train \
--workdir=/kaggle/working/gnnepcsaft --config=gnnepcsaft/configs/default.py \
--dataset=$DATASET --tags="parallel" \
--num_cpu=1.0 --num_cpu_trainer=0.33 --num_gpus=0.33 --num_workers=3 \
--config.model=PNAL --config.batch_size=512 \
--config.job_type="train" --framework="ray" --verbose=2

Loading .env environment variables...


I0519 01:40:20.297052 132708331857728 train.py:510] Calling train and evaluate!


2024-05-19 01:40:23,509	INFO worker.py:1749 -- Started a local Ray instance.


2024-05-19 01:40:24,653	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `<FrameworkTrainer>(...)`.


(RayTrainWorker pid=911) Setting up process group for: env:// [rank=0, world_size=3]


...

(RayTrainWorker pid=911) Epoch 37499, global step 150000: 'mape_den' reached 0.01797 (best 0.01797), saving model to '/kaggle/working/gnnepcsaft/train/checkpoints/model8_0-epoch=37499-mape_den=0.0180.ckpt' as top 1


(RayTrainWorker pid=911) Epoch 37499, global step 150000: 'train_mape' was not in top 1


(RayTrainWorker pid=912) Epoch 29999, global step 120000: 'train_mape' was not in top 1


(RayTrainWorker pid=913) Epoch 34999, global step 140000: 'train_mape' was not in top 1


Training on iteration 1. Current time: 2024-05-19 12:25:22. Total running time: 10hr 44min 57s


### Updating kaggle dataset

In [ ]:
%%bash
# !kaggle datasets metadata -p /kaggle/working/gnnepcsaft/train/checkpoints wildsonbblima/gnnepcsaft
# shutil.copy("gnnepcsaft/train/checkpoints/last_checkpoint.pth", f"gnnepcsaft/train/checkpoints/{model_name}-{model_suffix}e6.pth")
# os.remove("gnnepcsaft/train/checkpoints/last_checkpoint.pth")
# !kaggle datasets version -p /kaggle/working/gnnepcsaft/train/checkpoints -m "{model_name} at {model_suffix}e6 steps"

echo "*****  Push to data repo  *****"
pipenv run dvc add gnnepcsaft/train/checkpoints
pipenv run dvc push

git config --global user.email $USER_EMAIL
git config --global user.name $KAGGLE_USERNAME

echo "*****  Push to code repo  *****"
git pull
git commit gnnepcsaft/train/checkpoints.dvc -m "$MODEL_NAME at $MODEL_SUFFIX e6 steps"
git push

echo "*****  Finished  *****"

### Clear workspace

In [ ]:
!rm -rf * .*